<a href="https://colab.research.google.com/github/avimittal30/Azure-services-repo/blob/main/Agentic_rag_langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
!pip install langchain
!pip install langchain-openai
!pip install langchain-community
!pip install langgraph
!pip install langchain-chroma

  Using cached openai-1.58.1-py3-none-any.whl.metadata (27 kB)
Using cached openai-1.58.1-py3-none-any.whl (454 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.55.3
    Uninstalling openai-1.55.3:
      Successfully uninstalled openai-1.55.3


In [48]:
!pip install openai==1.55.3
!pip install httpx==0.27.2
!pip install typing-extensions>=4.11.0,<5.0.0

  Using cached openai-1.55.3-py3-none-any.whl.metadata (24 kB)
Using cached openai-1.55.3-py3-none-any.whl (389 kB)
  Attempting uninstall: openai
    Found existing installation: openai 1.58.1
    Uninstalling openai-1.58.1:
      Successfully uninstalled openai-1.58.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-openai 0.2.14 requires openai<2.0.0,>=1.58.1, but you have openai 1.55.3 which is incompatible.
/bin/bash: line 1: 5.0.0: No such file or directory


In [49]:
from google.colab import userdata
TAVILY_API_KEY=userdata.get('TAVILY_API_KEY')
OPENAI_KEY=userdata.get('OPENAI_API_KEY')

In [50]:
import os
os.environ['OPENAI_API_KEY'] = OPENAI_KEY
os.environ['TAVILY_API_KEY'] = TAVILY_API_KEY

In [51]:
from langchain_openai import OpenAIEmbeddings
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

In [52]:
!gdown 1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW

Downloading...
From (original): https://drive.google.com/uc?id=1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW
From (redirected): https://drive.google.com/uc?id=1oWBnoxBZ1Mpeond8XDUSO6J9oAjcRDyW&confirm=t&uuid=baf0b38c-a9f1-4d74-bc18-e86af5d10d5e
To: /content/simplewiki-2020-11-01.jsonl.gz
100% 50.2M/50.2M [00:00<00:00, 190MB/s]


In [131]:
import gzip
import json
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
wikipedia_filepath = 'simplewiki-2020-11-01.jsonl.gz'
docs = []

In [132]:
os.getcwd()

'/content'

In [133]:
import gzip
import json

# Path to the gzipped JSONL file
file_path = "simplewiki-2020-11-01.jsonl.gz"

# Counter for articles
article_count = 0

# Open the gzipped file and process it line by line
with gzip.open(file_path, 'rt', encoding='utf-8') as f:
    for line in f:
        # Each line is a JSON object representing an article
        article = json.loads(line.strip())
        article_count += 1
print(article_count)

169597


In [134]:
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:

        data = json.loads(line.strip())
        #Add documents
        docs.append({
                        'metadata': {
                                        'title': data.get('title'),
                                        'article_id': data.get('id')
                        },
                        'data': ' '.join(data.get('paragraphs')[0:3])
        # restrict data to first 3 paragraphs to run later modules faster
        })

In [135]:
len(docs)


169597

In [136]:
docs = [doc for doc in docs for x in ['smuggling'] if x in doc['data'].lower().split()]

In [137]:
docs

[{'metadata': {'title': 'Jihad Ahmed Jibril', 'article_id': '55144'},
  'data': 'Mohammed Jihad Ahmed Jibril (March 3, 1961 in Damascus - May 20, 2002 in Beirut) was the son of Ahmed Jibril, founder of the Popular Front for the Liberation of Palestine-General Command (PFLP-GC). He was killed by a car bomb. He was assassinated on May 20, 2002. Jihad Ahmed Jibril was the leader of the military wing of the PFLP-GC and was the heir apparent of the organization before he was killed in a car bombing in Beirut in 2002. The assassination was blamed on Israel, although Jibril had many other enemies, including the Lebanese Christian militia and other Palestinian nationalist organizations. Ahmed Jibril was co-leader of the militant Popular Front for the Liberation of Palestine-General Command. Israeli radio reports said that the son, Jihad Ahmed Jibril, was involved in smuggling arms and explosives to Palestinian militants in the West Bank and Gaza Strip.'},
 {'metadata': {'title': 'Crime in Russ

In [138]:
docs = [Document(page_content=doc['data'],
                 metadata=doc['metadata']) for doc in docs]

In [139]:
docs[:5]

[Document(metadata={'title': 'Jihad Ahmed Jibril', 'article_id': '55144'}, page_content='Mohammed Jihad Ahmed Jibril (March 3, 1961 in Damascus - May 20, 2002 in Beirut) was the son of Ahmed Jibril, founder of the Popular Front for the Liberation of Palestine-General Command (PFLP-GC). He was killed by a car bomb. He was assassinated on May 20, 2002. Jihad Ahmed Jibril was the leader of the military wing of the PFLP-GC and was the heir apparent of the organization before he was killed in a car bombing in Beirut in 2002. The assassination was blamed on Israel, although Jibril had many other enemies, including the Lebanese Christian militia and other Palestinian nationalist organizations. Ahmed Jibril was co-leader of the militant Popular Front for the Liberation of Palestine-General Command. Israeli radio reports said that the son, Jihad Ahmed Jibril, was involved in smuggling arms and explosives to Palestinian militants in the West Bank and Gaza Strip.'),
 Document(metadata={'title': '

In [140]:
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
chunked_docs = splitter.split_documents(docs)
chunked_docs[:3]

[Document(metadata={'title': 'Jihad Ahmed Jibril', 'article_id': '55144'}, page_content='Mohammed Jihad Ahmed Jibril (March 3, 1961 in Damascus - May 20, 2002 in Beirut) was the son of Ahmed Jibril, founder of the Popular Front for the Liberation of Palestine-General Command (PFLP-GC). He was killed by a car bomb. He was assassinated on May 20, 2002. Jihad Ahmed Jibril was the leader of the military wing of the PFLP-GC and was the heir apparent of the organization before he was killed in a car bombing in Beirut in 2002. The assassination was blamed on Israel, although Jibril had many other enemies, including the Lebanese Christian militia and other Palestinian nationalist organizations. Ahmed Jibril was co-leader of the militant Popular Front for the Liberation of Palestine-General Command. Israeli radio reports said that the son, Jihad Ahmed Jibril, was involved in smuggling arms and explosives to Palestinian militants in the West Bank and Gaza Strip.'),
 Document(metadata={'title': '

In [144]:
from langchain_chroma import Chroma
# create vector DB of docs and embeddings - takes < 30s on Colab
chroma_db = Chroma.from_documents(documents=chunked_docs,
                                  collection_name='rag_wikipedia_smuggling',
                                  embedding=openai_embed_model,
# need to set the distance function to cosine else it uses Euclidean by default
# check https://docs.trychroma.com/guides#changing-the-distance-function
                                  collection_metadata={"hnsw:space": "cosine"},
                                  persist_directory="./wikipedia_db_smug")

In [150]:
similarity_threshold_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold",
                       search_kwargs={"k": 5,
                       "score_threshold": 0.3})

In [296]:
# We can then test if our retriever is working on some sample queries.
query = "What is smuggling, and why do people engage in it?"
top3_docs = similarity_threshold_retriever.invoke(query)
[doc.page_content for doc in top3_docs]

['Smuggling means illegal transport of goods, objects substances or information across state, national or international borders. Smugglers do this for various reasons including avoiding tariffs.',
 'Arms trafficking or gun-running is smuggling of weapons and ammunition. Arms trafficking happens everywhere but it is a very big problem in places with violent political conflict or civil war. International organizations for human rights, including Amnesty International and Human Rights Watch are looking into fixing this problem.',
 'Rum-running, or bootlegging, is the illegal business of taking alcoholic beverages where they are against the law. It is usually done to evade taxation or prohibition laws. "Rum-running" usually means smuggling over water. The term is thought to come from the start of the Prohibition era in the United States. This is when ships from Bimini in the western Bahamas transported cheap rum to Florida to be sold illegally. But there was little profit in cheap rum. The

In [164]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI
# Data model for LLM output format
class GradeDocuments(BaseModel):
    # """Binary score for relevance check on retrieved documents."""
    binary_score: str = Field(
        # description="Documents are relevant to the question, 'yes' or 'no'"
    )
# LLM for grading
llm = ChatOpenAI(model="gpt-4o", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)
# Prompt template for grading
SYS_PROMPT = """You are an expert grader assessing relevance of a retrieved document to a user question.
                Follow these instructions for grading:
                  - If the document contains keyword(s) or semantic meaning related to the question, grade it as relevant.
                  - Your grade should be either 'yes' or 'no' to indicate whether the document is relevant to the question or not."""

grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        ("human", """Retrieved document:
                     {document}
                     User question:
                     {question}
                  """),
    ]
)

crude_response = grade_prompt | llm

In [165]:
for doc in top3_docs:
    print(doc.page_content)
    print('GRADE:', crude_response.invoke({"question": query,
                                       "document": doc.page_content}))

Smuggling means illegal transport of goods, objects substances or information across state, national or international borders. Smugglers do this for various reasons including avoiding tariffs.
GRADE: content='yes' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 2, 'prompt_tokens': 139, 'total_tokens': 141, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_d28bcae782', 'finish_reason': 'stop', 'logprobs': None} id='run-c9871944-8ff5-4758-8eac-44682091a294-0' usage_metadata={'input_tokens': 139, 'output_tokens': 2, 'total_tokens': 141, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
Arms trafficking or gun-running is smuggling of weapons and ammunition. Arms trafficking ha

In [ ]:
# Build grader chain
doc_grader = (grade_prompt
                  |
              structured_llm_grader)

In [179]:
query = "What is smuggling, and why do people engage in it?"
top3_docs = similarity_threshold_retriever.invoke(query)
# Storing the responses in a dictionary
structured_responses = {'content': [], 'score': []}
for doc in top3_docs:
    print(doc.page_content)

    structured_responses['content'].append(doc.page_content)
    structured_responses['score'].append(doc_grader.invoke({"question": query,"document": doc.page_content}).binary_score)

    print('GRADE:', doc_grader.invoke({"question": query,
                                       "document": doc.page_content}))
    print()

Smuggling means illegal transport of goods, objects substances or information across state, national or international borders. Smugglers do this for various reasons including avoiding tariffs.
GRADE: binary_score='yes'

Arms trafficking or gun-running is smuggling of weapons and ammunition. Arms trafficking happens everywhere but it is a very big problem in places with violent political conflict or civil war. International organizations for human rights, including Amnesty International and Human Rights Watch are looking into fixing this problem.
GRADE: binary_score='yes'

Rum-running, or bootlegging, is the illegal business of taking alcoholic beverages where they are against the law. It is usually done to evade taxation or prohibition laws. "Rum-running" usually means smuggling over water. The term is thought to come from the start of the Prohibition era in the United States. This is when ships from Bimini in the western Bahamas transported cheap rum to Florida to be sold illegally. B

In [191]:
filtered_responses = {key: [val for i, val in enumerate(structured_responses[key]) if structured_responses['score'][i] == 'yes'] for key in structured_responses}
filtered_responses['content']

['Smuggling means illegal transport of goods, objects substances or information across state, national or international borders. Smugglers do this for various reasons including avoiding tariffs.',
 'Arms trafficking or gun-running is smuggling of weapons and ammunition. Arms trafficking happens everywhere but it is a very big problem in places with violent political conflict or civil war. International organizations for human rights, including Amnesty International and Human Rights Watch are looking into fixing this problem.',
 'Rum-running, or bootlegging, is the illegal business of taking alcoholic beverages where they are against the law. It is usually done to evade taxation or prohibition laws. "Rum-running" usually means smuggling over water. The term is thought to come from the start of the Prohibition era in the United States. This is when ships from Bimini in the western Bahamas transported cheap rum to Florida to be sold illegally. But there was little profit in cheap rum. The

In [195]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
# Create RAG prompt for response generation
prompt = """You are an assistant for question-answering tasks.
            Use the following pieces of retrieved context to answer the question.
            If no context is present or if you don't know the answer, just say that you don't know the answer.
            Do not make up the answer unless it is there in the provided context.
            Give a detailed answer and to the point answer with regard to the question.
            Question:
            {question}
            Context:
            {context}
            Answer:
         """
prompt_template = ChatPromptTemplate.from_template(prompt)
# Initialize connection with GPT-4o
chatgpt = ChatOpenAI(model_name='gpt-4o', temperature=0)
# Used for separating context docs with new lines
def format_docs(docs):
    return "\n\n".join(doc for doc in docs)
# create QA RAG chain
qa_rag_chain = (
    {
        "context": (itemgetter('context')
                        |
                    RunnableLambda(format_docs)),
        "question": itemgetter('question')
    }
      |
    prompt_template
      |
    chatgpt
      |
    StrOutputParser()
)

In [196]:
query = "What is smuggling and why do people engage in it?"
top3_docs = similarity_threshold_retriever.invoke(query)
result = qa_rag_chain.invoke(
    {"context": filtered_responses['content'], "question": query}
)
print(result)

Smuggling refers to the illegal transport of goods, objects, substances, or information across state, national, or international borders. People engage in smuggling for various reasons, primarily to avoid tariffs, taxes, or legal restrictions imposed by governments. By bypassing these legal and financial barriers, smugglers can often sell goods at a higher profit margin.

There are different types of smuggling, each with its own motivations and methods:

1. **Arms Trafficking**: This involves the illegal smuggling of weapons and ammunition. It is particularly prevalent in regions experiencing violent political conflict or civil war. The demand for weapons in such areas drives the smuggling activities, despite international efforts by organizations like Amnesty International and Human Rights Watch to address the issue.

2. **Rum-running or Bootlegging**: This is the illegal transport of alcoholic beverages to evade taxation or prohibition laws. Historically, during the Prohibition era i

In [197]:
# LLM for question rewriting
llm = ChatOpenAI(model="gpt-4o", temperature=0)
# Prompt template for rewriting
SYS_PROMPT = """Act as a question re-writer and perform the following task:
                 - Convert the following input question to a better version that is optimized for web search.
                 - When re-writing, look at the input question and try to reason about the underlying semantic intent / meaning.
             """
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYS_PROMPT),
        ("human", """Here is the initial question:
                     {question}
                     Formulate an improved question.
                  """,
        ),
    ]
)

# Create rephraser chain
question_rewriter = (re_write_prompt
                        |
                       llm
                        |
                     StrOutputParser())

In [ ]:
query = "who won the champions league in 2024?"
question_rewriter.invoke({"question": query})

'Who was the winner of the UEFA Champions League in 2024?'

In [207]:
from langchain_community.tools.tavily_search import TavilySearchResults
tv_search = TavilySearchResults(max_results=3, search_depth='advanced',max_tokens=10000)

In [276]:
from typing import List
from typing_extensions import TypedDict
class GraphState(TypedDict):
    # """
    # Represents the state of our graph.
    # Attributes:
    #     question: question
    #     generation: LLM response generation
    #     web_search_needed: flag of whether to add web search - yes or no
    #     documents: list of context documents
    # """
    question: str
    generation: str
    web_search_needed: str
    documents: List[str]

In [297]:
def retrieve(state):
    """
    Retrieve documents
    Args:
        state (dict): The current graph state
    Returns:
        state (dict): New key added to state, documents - that contains retrieved context documents
    """
    print("---RETRIEVAL FROM VECTOR DB---")
    question = state["question"]
    # Retrieval
    documents = similarity_threshold_retriever.invoke(question)
    return {"documents": documents, "question": question}

In [278]:
def grade_documents(state):
    print ("---- Inside grade_documents --------------------")
    # """
    # Determines whether the retrieved documents are relevant to the question
    # by using an LLM Grader.
    # If any document are not relevant to question or documents are empty - Web Search needs to be done
    # If all documents are relevant to question - Web Search is not needed
    # Helps filtering out irrelevant documents
    # Args:
    #     state (dict): The current graph state
    # Returns:
    #     state (dict): Updates documents key with only filtered relevant documents
    #"""
    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]
    # Score each doc
    filtered_docs = []
    web_search_needed = "No"
    if documents:
        for d in documents:
            score = doc_grader.invoke(
                {"question": question, "document": d.page_content}
            )
            grade = score.binary_score
            if grade == "yes":
                print("---GRADE: DOCUMENT RELEVANT---")
                filtered_docs.append(d.page_content)

    elif len(filtered_docs)==0:
        print("---NO DOCUMENTS RETRIEVED---")
        web_search_needed = "Yes"
    return {"documents": filtered_docs, "question": question,
            "web_search_needed": web_search_needed}

In [279]:
def rewrite_query(state):
    # """
    # Rewrite the query to produce a better question.
    # Args:
    #     state (dict): The current graph state
    # Returns:
    #     state (dict): Updates question key with a re-phrased or re-written question
    # """
    print("---REWRITE QUERY---")
    question = state["question"]
    documents = state["documents"]
    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}

In [298]:
from langchain.schema import Document
def web_search(state):
    print("--- Inside Web Search ------------")
    # """
    # Web search based on the re-written question.
    # Args:
    #     state (dict): The current graph state
    # Returns:
    #     state (dict): Updates documents key with appended web results
    # """
    print("---WEB SEARCH---")
    question = state["question"]

    documents = state["documents"]

    # Web search
    docs = tv_search.invoke(question)
    web_results = "\n\n".join([d["content"] for d in docs])
    # web_results = Document(page_content=web_results)
    documents.append(web_results)

    return {"documents": documents, "question": question}

In [302]:
def gen_resp(state):
    print("--- Inside Generate Answer ------------")
    # """
    # Generate answer from context document using LLM
    # Args:
    #     state (dict): The current graph state
    # Returns:
    #     state (dict): New key added to state, generation, that contains LLM generation
    # """
    # print("---GENERATE ANSWER---")
    question = state["question"]
    documents = state["documents"]
    # RAG generation
    generation = qa_rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question,
            "generation": generation}

In [304]:
def decide_context_source(state):
    print("--- Inside Decide to generate ------------")
    # """
    # Determines whether to generate an answer, or re-generate a question.
    # Args:
    #     state (dict): The current graph state
    # Returns:
    #     str: Binary decision for next node to call
    # """
    # print("---ASSESS GRADED DOCUMENTS---")
    web_search_needed = state["web_search_needed"]
    print(f'Web Search Needed?: {web_search_needed}')
    if web_search_needed == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        return "web_search"
        print("web_search")
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE RESPONSE---")
        return "gen_resp"

In [306]:
from langgraph.graph import END, StateGraph
agentic_rag = StateGraph(GraphState)
# Define the nodes
agentic_rag.add_node("retrieve", retrieve)  # retrieve
agentic_rag.add_node("grade_documents", grade_documents)  # grade documents
agentic_rag.add_node("web_search", web_search)  # web search
agentic_rag.add_node("gen_resp", gen_resp)  # generate answer
# Build graph
agentic_rag.set_entry_point("retrieve")
agentic_rag.add_edge("retrieve", "grade_documents")
agentic_rag.add_conditional_edges(
    "grade_documents",
    decide_context_source,
)

agentic_rag.add_edge("web_search", "gen_resp")
agentic_rag.add_edge("gen_resp", END)
# Compile
agentic_rag = agentic_rag.compile()

In [308]:
query = "What is smuggling and what motivates people to engage in smuggling?"
response = agentic_rag.invoke({"question": query})
print(response['generation'])


---RETRIEVAL FROM VECTOR DB---
---- Inside grade_documents --------------------
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
--- Inside Decide to generate ------------
Web Search Needed?: No
---DECISION: GENERATE RESPONSE---
--- Inside Generate Answer ------------
Smuggling refers to the illegal transportation of goods, objects, substances, or information across state, national, or international borders. This activity is often undertaken to avoid tariffs, taxes, or legal restrictions imposed by governments. Smuggling can involve a wide range of items, including weapons, alcohol, drugs, and even information.

There are several motivations for people to engage in smuggling:

1. **Economic Gain**: The primary motivation for smuggling is often financial profit. By evading taxes and tariffs, smugglers can sell goods at a lower price or pocket the difference, leading to significant profits. For example

In [ ]:
response['generation']

'The capital city of India is New Delhi. It is situated in the northern part of the country and serves as the political, cultural, and administrative center of India. New Delhi was officially declared the capital of India in 1911 during British rule, replacing Kolkata, and continued to be the capital after India gained independence on August 15, 1947.'

In [274]:
query = "who won the t20 cricket worldcup in 2024?"
response = agentic_rag.invoke({"question": query})
print(response['generation'])

---- Inside grade_documents --------------------
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---NO DOCUMENTS RETRIEVED---
--- Inside Decide to generate ------------
Web Search Needed?: Yes
--- Inside Web Search ------------
---WEB SEARCH---
--- Inside Generate Answer ------------
India won the Men's T20 World Cup in 2024. They defeated South Africa by seven runs in the final held in Barbados to claim their second title. South Africa was denied their first World Cup title after this heart-breaking defeat at Kensington Oval.


In [ ]:
response

{'question': 'Who was the winner of the UEFA Champions League in 2024?',
 'generation': 'The winner of the UEFA Champions League in 2024 was Real Madrid. They defeated Borussia Dortmund in the final, which took place at Wembley Stadium in London, England, on June 1, 2024. Goals from Dani Carvajal and Vinicius Junior secured the victory for Real Madrid.',
 'web_search_needed': 'Yes',
 'documents': [Document(metadata={}, page_content='2024 UEFA Champions League final - Simple English Wikipedia, the free encyclopedia 2024 UEFA Champions League final 2024 UEFA Champions League final The 2024 UEFA Champions League final was the last match of the 2023–24 UEFA Champions League, which is a big football competition in Europe. This final happened at Wembley Stadium in London, England, on June 1, 2024.[5][6] The team that won, Real Madrid, gets to play against the winners of the 2023–24 UEFA Europa League, Atalanta, in the 2024 UEFA Super Cup. Match ↑ "Official Champions League final PlayStation 

In [ ]:
from IPython.display import Image, display, Markdown
display(Markdown(response['generation']))

The winners of the UEFA Champions League in 2024 were Borussia Dortmund, who defeated their opponents 2-0 in the final held at Wembley Stadium, London, England, on June 1, 2024.